In [4]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";  # The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  # Do other imports now...

import tensorflow as tf
    
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

from datetime import datetime
from keras.models import Model
from keras.layers import Input, Dense, Dot, Add, Flatten, Concatenate, Embedding, LSTM, GRU, Input, Dense, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook, tqdm
tqdm.pandas()
import gc
from collections import Counter

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, recall_score, precision_score, mean_squared_error
import json

In [16]:
import random
seed_value = 42
def init_seeds():
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    
init_seeds()

# Reading The Family-Intervals Dict

In [1]:
def read_dict(name):
    with open(name, 'r') as f:
        return json.load(f)
    
family_intervals = read_dict('../Family_Intervals/intervals.json')

for key in family_intervals.keys():
    if key != 'other':
        lower, upper = family_intervals[key]
        family_intervals[key] = range(lower, upper + 1)

index_to_name = read_dict('../Family_Intervals/Family_Indexes.json')
index_to_interval = []
for key in index_to_name.keys():
    index_to_interval += [family_intervals[index_to_name[key]]]
index_to_interval

# Selecting Top n individuals

In [18]:

def get_mask(family,indlength=480):
    family_intervals = index_to_interval
    mask = np.array([0] * indlength)

    for family_mask_index in range(len(family)):
        family_interval = family_intervals[family_mask_index]
        for feature_index in family_interval:
            mask[feature_index] = family[family_mask_index]
    
    return mask

# returns all unique inds that were present in an expr
def get_all_inds(pop_dict, with_family=False):
    all_inds = set()
    gens = list(pop_dict.keys())
    for gen_number in gens:
        pop_gen = pop_dict[gen_number]
        if with_family:
            pop_gen = [tuple([tuple(i[0]), tuple(i[1])]) for i in pop_gen]
        else:
            pop_gen = [tuple(i) for i in pop_gen]
        all_inds.update(pop_gen)
    
    return all_inds

def get_top_n_from_gens(pop_dict, fitness_dict, n, top_biggest = False, gen=None, with_family=False):
    inds = None
    
    if gen is None:
        inds = get_all_inds(pop_dict, with_family)
    else:
        inds = list(map(lambda i:tuple(i), pop_dict[gen]))
        
    inds = list(inds)
    inds.sort(key =  lambda ind: fitness_dict[str(ind)] if str(ind) in fitness_dict else 0)
    
    if top_biggest:
        inds.reverse()
    
    desired_inds = inds[0:n]
    inds_metric = list(map(lambda ind : fitness_dict[str(ind)] if str(ind) in fitness_dict else 0, desired_inds))
    
    
    if with_family:
        desired_inds = [np.multiply(np.array(i[0]), np.array(get_mask(i[1]))) for i in desired_inds]
        
    return desired_inds, inds_metric

def get_top_n_ind_from_expr(folder, n, top_biggest = False, gen=None, with_family=False):
    fitness_dict = read_dict(folder + "fitness_dict.json")
    pop_dict = read_dict (folder + "gens_dict.json")
    
    key_func = lambda ind: fitness_dict[str(ind)]
    return get_top_n_from_gens(pop_dict, fitness_dict, n, top_biggest = top_biggest, gen=gen, with_family=with_family)
    

#### Top individual from the only-pred experiment

In [20]:
top_inds_only_predict,top_metric = get_top_n_ind_from_expr(
    folder = '../Results/GA_RES_17_4_TOURNAMNET_ONLY_PRED/' ,n = 1 ,top_biggest = True, with_family=False)

best_only_predict = top_inds_only_predict[0]

#### Top individual from the full-train experiment

In [24]:
top_inds_full_train,top_metric2 = get_top_n_ind_from_expr(
    folder = '../Results/GA_RES_17_4_TOURNAMNET_FULL_TRAIN/' ,n = 1,top_biggest = True, with_family=False)
best_full_train = top_inds_full_train[0]

# Loading Contextual Information For Testing

In [30]:
source_column = "userId"
target_column = "triggerValue"

In [31]:
import numpy as np

from sklearn.model_selection import train_test_split

def get_train_test_val_split(arr):
    # train = 70%
    # test = 20% 
    # val = 10%
    x_train, x_test = train_test_split(arr, test_size=0.3 , shuffle=False)
    x_val ,x_test = train_test_split(x_test, test_size=2/3 , shuffle = False)
    return x_train, x_test, x_val

In [32]:
y_train, y_test, y_val = np.load('../data/y_train.npy'), np.load('../data/y_test.npy'),np.load('../data/y_val.npy')
item_train, item_test, item_val = np.load('../data/item_train.npy'),np.load('../data/item_test.npy'),np.load('../data/item_val.npy')
user_train, user_test, user_val = np.load('../data/user_train.npy'),np.load('../data/user_test.npy'),np.load('../data/user_val.npy')

In [34]:
def reduce_features(ind, arr):
    res = []
    for i in range(len(ind)):
        if ind[i] == 1:
            res += [arr[i]]
    return np.array(res)


def reduce_by_ind(x_arr, ind):
    return np.array(list(map(lambda row : reduce_features(ind, row) , x_arr)))


In [35]:
context_train, context_test, context_val = np.load('../data/context_train.npy'),np.load('../data/context_test.npy'),np.load('../data/context_val.npy')
pca_context_train, pca_context_test, pca_context_val = np.load('../data/context_train_pca.npy'),np.load('../data/context_test_pca.npy'),np.load('../data/context_val_pca.npy')
autoenc_context_train, autoenc_context_test, autoenc_context_val = np.load('../data/context_train_autoenc.npy'),np.load('../data/context_test_autoenc.npy'),np.load('../data/context_val_autoenc.npy')

assert len(context_test) == len(pca_context_test) and len(context_test) == len(autoenc_context_test)

In [36]:
# Indexes for CAMF Feature (weather etc..)
baseline_indexes = [6, 7, 8, 9, 10, 11]

def reduce_by_indexes(arr, indexes):
    res = []
    for ind in indexes:
        res += [arr[ind]]
    return np.array(res)

def map_reduce_indexes(arr, indexes):
    return np.array(list(map(lambda element: reduce_by_indexes(element, indexes), arr)))

baseline_train = map_reduce_indexes(context_train, baseline_indexes)
baseline_test = map_reduce_indexes(context_test, baseline_indexes)
baseline_val  = map_reduce_indexes(context_val, baseline_indexes)

assert len(baseline_train[0]) == len(baseline_indexes)

In [37]:
feed_forward_indexes = np.load('../feature_selections/CARS_RESULTS/feed_forward/feed_forward.npy')

feed_forward_train = map_reduce_indexes(context_train, feed_forward_indexes)
feed_forward_test = map_reduce_indexes(context_test, feed_forward_indexes)
feed_forward_val  = map_reduce_indexes(context_val, feed_forward_indexes)

assert len(feed_forward_indexes) == len(feed_forward_test[0])

In [38]:
BE_indexes = np.load('../feature_selections/CARS_RESULTS/backwards_elimination.npy')

BE_train = map_reduce_indexes(context_train, BE_indexes)
BE_test = map_reduce_indexes(context_test, BE_indexes)
BE_val  = map_reduce_indexes(context_val, BE_indexes)

assert len(BE_indexes) == len(BE_train[0])
len(BE_indexes)

478

In [39]:
onlypred_context_train = reduce_by_ind(context_train, best_only_predict)
onlypred_context_test = reduce_by_ind(context_test, best_only_predict)
onlypred_context_val = reduce_by_ind(context_val, best_only_predict)

assert len(onlypred_context_train) == len(context_train) and len(onlypred_context_val) == len(context_val) and len(onlypred_context_test) == len(context_test)
assert len(onlypred_context_train[0]) == sum(best_only_predict)

In [40]:
fulltrain_context_train = reduce_by_ind(context_train, best_full_train)
fulltrain_context_test = reduce_by_ind(context_test, best_full_train)
fulltrain_context_val = reduce_by_ind(context_val, best_full_train)

assert len(fulltrain_context_train) == len(context_train) and len(fulltrain_context_val) == len(context_val) and len(fulltrain_context_test) == len(context_test)
assert len(fulltrain_context_train[0]) == sum(best_full_train)

In [41]:
lstm_autoencoder_3_train = np.load('../data/LSTM/LSTM_AUTO_ENCODER_3_x_train.npy')
lstm_autoencoder_3_test = np.load('../data/LSTM/LSTM_AUTO_ENCODER_3_x_test.npy')
lstm_autoencoder_3_val = np.load('../data/LSTM/LSTM_AUTO_ENCODER_3_x_val.npy')

assert len(context_train) == len(lstm_autoencoder_3_train) and len(context_val) == len(lstm_autoencoder_3_val) and len(context_test) == len(lstm_autoencoder_3_test)

In [42]:
lstm_autoencoder_4_train = np.load('../data/LSTM/LSTM_AUTO_ENCODER_4_x_train.npy')
lstm_autoencoder_4_test = np.load('../data/LSTM/LSTM_AUTO_ENCODER_4_x_test.npy')
lstm_autoencoder_4_val = np.load('../data/LSTM/LSTM_AUTO_ENCODER_4_x_val.npy')

assert len(context_train) == len(lstm_autoencoder_4_train) and len(context_val) == len(lstm_autoencoder_4_val) and len(context_test) == len(lstm_autoencoder_4_test)

In [43]:
lstm_autoencoder_5_train = np.load('../data/LSTM/LSTM_AUTO_ENCODER_5_x_train.npy')
lstm_autoencoder_5_test = np.load('../data/LSTM/LSTM_AUTO_ENCODER_5_x_test.npy')
lstm_autoencoder_5_val = np.load('../data/LSTM/LSTM_AUTO_ENCODER_5_x_val.npy')

assert len(context_train) == len(lstm_autoencoder_5_train) and len(context_val) == len(lstm_autoencoder_5_val) and len(context_test) == len(lstm_autoencoder_5_test)

In [45]:
context_dimensions = len(context_train[0])
context_dimensions

480

In [2]:
user_train

In [3]:
source_unique = max([max(user_test), max(user_train), max(user_val)]) + 1
print(source_unique)

target_unique = max([max(item_test), max(item_train), max(item_val)]) + 1
print(target_unique)


# Setting-up the contextual modeling architecture

In [48]:
from keras.layers import Dropout
from keras.layers import BatchNormalization

default_info = {'final_activation':'sigmoid', 'loss':'binary_crossentropy',
                     'dropout' : True , 'batch_norm' : True
                   }

def get_full_arch(context_dim, info=None):
    global default_info
    
    if info is None:
        info = default_info
    
    loss = info['loss']
    final_activation = info['final_activation']
    BATCH_NORM_MLP = info['batch_norm']
    DROPOUT_ENS_LAYER = info['dropout']
    
    
    embedding_dim = 16
    print("FULL ARCH {}, emb_dim = {}, loss = {}, activation = {}".format(context_dim, embedding_dim, loss, final_activation))
    input_target = Input(shape=[1], name="input_item")
    input_source = Input(shape=[1], name="input_user")
    input_context = Input(shape=[context_dim], name="input_context")

    target_emb = Embedding(input_dim=target_unique, output_dim=embedding_dim,
                           input_length=1, name='target_emb')(input_target)
    source_emb = Embedding(input_dim=source_unique, output_dim=embedding_dim,
                           input_length=1, name='source_emb')(input_source)

    target_emb_mlp = Embedding(input_dim=target_unique, output_dim=embedding_dim,
                              input_length=1, name='target_emb_mlp')(input_target)
    source_emb_mlp = Embedding(input_dim=source_unique, output_dim=embedding_dim,input_length=1, name='source_emb_mlp')(input_source)

    # adding bias
    target_bias = Embedding(input_dim=target_unique, output_dim=1,
                            input_length=1, name='target_bias')(input_target)
    source_bias = Embedding(input_dim=source_unique, output_dim=1,
                            input_length=1, name='source_bias')(input_source)

    target_flat = Flatten()(target_emb)
    source_flat = Flatten()(source_emb)

    target_flat_mlp = Flatten()(target_emb_mlp)
    source_flat_mlp = Flatten()(source_emb_mlp)

    concat = Concatenate()([target_flat_mlp, source_flat_mlp, input_context])

    target_flat_bias = Flatten()(target_bias)
    source_flat_bias = Flatten()(source_bias)
    
    mlp_dense_input = Dense(200, name='mlp_dense_in', activation='relu')(concat)
    b0 = BatchNormalization()(mlp_dense_input)
    mlp_hidden_0 = Dense(100, name='mlp_hidden_0', activation='relu')(b0)

    b1 = BatchNormalization()(mlp_hidden_0)
    mlp_hidden_1 = Dense(50, name='mlp_hidden_1', activation='relu')(b1)
    b2 = BatchNormalization()(mlp_hidden_1)    
    mlp_dense_out = Dense(1, name='mlp_dense_out', activation='sigmoid')(b2)
         

    dot = Dot(axes=1, name='dot')([source_flat, target_flat])

    add = Add()([dot, target_flat_bias, source_flat_bias])

    # creating a 1*2 vector that is a concatenation between the two sides
    concat_left_and_right = Concatenate()([add, mlp_dense_out])
    
    # I used this for the best results so far
    concat_left_and_right = Dropout(rate = 0.5, seed = 42)(concat_left_and_right)
    
    last_layers_neurons = 6
    print(f"last layer neurons = {last_layers_neurons}")
    final_mlp_dense_input = Dense(last_layers_neurons, name='final_dense_in', activation = 'relu')(concat_left_and_right)
    final_mlp_dense_out =  Dense(1, name='final_dense_out', activation=final_activation)(final_mlp_dense_input)

    # output = Activation('sigmoid')(add)

    model = Model(inputs=[input_target, input_source, input_context], outputs=final_mlp_dense_out)

    model.compile(optimizer='adam', loss=loss, metrics=['mae'])

    
    return model

In [49]:
def get_full_arch_no_context():
    embedding_dim = 16
    print("FULL ARCH NO CONTEXT _ emb dim = {}".format(embedding_dim))

    
    input_target = Input(shape=[1], name="input_item")
    input_source = Input(shape=[1], name="input_user")

    target_emb = Embedding(input_dim=target_unique, output_dim=embedding_dim,
                           input_length=1, name='target_emb')(input_target)
    source_emb = Embedding(input_dim=source_unique, output_dim=embedding_dim,
                           input_length=1, name='source_emb')(input_source)

    target_emb_mlp = Embedding(input_dim=target_unique, output_dim=embedding_dim,
                              input_length=1, name='target_emb_mlp')(input_target)
    source_emb_mlp = Embedding(input_dim=source_unique, output_dim=embedding_dim,input_length=1, name='source_emb_mlp')(input_source)

    # adding bias
    target_bias = Embedding(input_dim=target_unique, output_dim=1,
                            input_length=1, name='target_bias')(input_target)
    source_bias = Embedding(input_dim=source_unique, output_dim=1,
                            input_length=1, name='source_bias')(input_source)

    target_flat = Flatten()(target_emb)
    source_flat = Flatten()(source_emb)

    target_flat_mlp = Flatten()(target_emb_mlp)
    source_flat_mlp = Flatten()(source_emb_mlp)

    concat = Concatenate()([target_flat_mlp, source_flat_mlp])

    target_flat_bias = Flatten()(target_bias)
    source_flat_bias = Flatten()(source_bias)
    
    mlp_dense_input = Dense(100, name='mlp_dense_in', activation='relu')(concat)
    mlp_hidden_1 = Dense(50, name='mlp_hidden_1', activation='relu')(mlp_dense_input)
    mlp_dense_out = Dense(1, name='mlp_dense_out', activation='sigmoid')(mlp_hidden_1)
         

    dot = Dot(axes=1, name='dot')([source_flat, target_flat])

    add = Add()([dot, target_flat_bias, source_flat_bias])

    # creating a 1*2 vector that is a concatenation between the two sides
    concat_left_and_right = Concatenate()([add, mlp_dense_out])
    
    final_mlp_dense_input = Dense(3, name='final_dense_in', activation = 'relu')(concat_left_and_right)
    final_mlp_dense_out =  Dense(1, name='final_dense_out', activation='sigmoid')(final_mlp_dense_input)

    model = Model(inputs=[input_target, input_source], outputs=final_mlp_dense_out)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mae'])

    
    return model

In [50]:
def init_metric_dict():
    metric_dict = {}
    metric_dict["auc_vals"] = []
    metric_dict["auc_tests"] = []
    metric_dict["mse_vals"] = []
    metric_dict["mse_tests"] = []
    metric_dict["logloss_vals"] = []
    metric_dict["logloss_tests"] = []
    metric_dict["y_val_pred"] = []
    metric_dict["y_test_pred"] = []
    return metric_dict

In [51]:
from sklearn.metrics import log_loss

verbose = 0

def evaluate_model(model, fresh_weights_path, x_train, x_val, x_test, test_name):
    print("evaluating")
    model.load_weights(fresh_weights_path)
    
    best_arch_path = '../data/best_arch_{}.h5'.format(test_name)
    best_adam_weights = '../data/best_adam_weights_{}.h5'.format(test_name)
    
    es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='min')
    mcp = ModelCheckpoint(best_arch_path , save_best_only = True, mode = 'min')
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mae'])
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), 
                        epochs=100, batch_size=256, verbose=verbose, callbacks=[es,mcp])
    
    model = load_model(best_arch_path)
    model.save_weights(best_adam_weights)
    
    model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['mae'])
    model.load_weights(best_adam_weights)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), 
                        epochs=100, batch_size=256, verbose=verbose, callbacks=[es,mcp])
    
    model = load_model(best_arch_path)
    
    
    val_predict = model.predict(x_val, batch_size=256)
    
    auc_val = roc_auc_score(y_val, val_predict)
    mse_val = mean_squared_error(y_val, val_predict)
    logloss_val = log_loss(y_val , val_predict)
    
    
    test_predict = model.predict(x_test, batch_size=256)
    
    auc_test = roc_auc_score(y_test, test_predict)
    mse_test = mean_squared_error(y_test, test_predict)
    logloss_test = log_loss(y_test, test_predict)
    
    metric_dict = init_metric_dict()
    
    del es
    del mcp
    
    metric_dict["auc_vals"] = auc_val
    metric_dict["auc_tests"] = auc_test
    metric_dict["mse_vals"] = mse_val
    metric_dict["mse_tests"] = mse_test
    metric_dict["logloss_vals"] = logloss_val
    metric_dict["logloss_tests"] = logloss_test
    metric_dict["y_val_pred"] = val_predict
    metric_dict["y_test_pred"] = test_predict
        
    return metric_dict



In [52]:
def sample_model(n_samples, eval_func):
    metric_dict = init_metric_dict()
    
    for i in range(n_samples):    
        metric_dict_current = eval_func()
        
        for key in metric_dict.keys():
            metric_dict[key] += [metric_dict_current[key]]
            
    
    return metric_dict

In [53]:
def arch_testing_wrapper(args):
    return start_arch_testing(*args)

def start_arch_testing(con_train, con_test, con_val, samples=5, model_type="FULL", info = None,name_=""):
    init_seeds()
    dim = len(con_train[0])
    weights_path = '../data/BEST_WEIGHTS_{}'.format(name_)
    global default_info
    
    if info is None:
        info = default_info
    else:
        for key in default_info.keys():
            if key not in info:
                info[key] = default_info[key]

    x_train = [item_train, user_train]
    x_val = [item_val, user_val]
    x_test = [item_test, user_test]
    
    if model_type == "FULL":
        model = get_full_arch(dim, info)
        x_train += [con_train]
        x_val += [con_val]
        x_test += [con_test]
        
    elif model_type == "MLP":
        model = get_simple_mlp(dim)
        x_train = con_train
        x_val = con_val
        x_test = con_test
    
    elif model_type == "RIGHT":
        model = get_right_side(dim)
        x_train += [con_train]
        x_val += [con_val]
        x_test += [con_test] 
        
    elif model_type == "LEFT":
        model = get_left_side()
    
    elif model_type == 'FULL_NO_CONTEXT':
        model = get_full_arch_no_context()
    
    elif model_type == 'EXTENDED':
        dims = [len(x[0]) for x in con_train]
        model = get_full_arch_extended(dims)
        x_train += con_train
        x_test += con_test
        x_val += con_val
        print([len(z) for z in x_train])
        

    model.save_weights(weights_path)
    return sample_model(samples, lambda : evaluate_model(model, weights_path, x_train = x_train, x_val = x_val, x_test = x_test, test_name=name_))    

In [54]:
samp = 1
m_type = "FULL"

# Running the models

In [56]:
name_vanilla = 'All-Context-No-Reduction'
name_auto = 'Latent-AE'
name_pca = 'Latent-PCA'
name_full_train = 'BestInd-Full-Train'
name_only_pred = 'OnlyPred'
name_user_item = 'NeuMF'
name_baseline = "Explicit (CAMF)"
name_extended = "EXTENDED NeuMF"
name_feed_forward = "FF Feature selection"
name_BE = "BE Feature Selection"
name_LSTM_3 = "LSTM Autoencoder (3)"
name_LSTM_4 = "LSTM Autoencoder (4)"
name_LSTM_5 = "LSTM Autoencoder (5)"


torun=[
    (context_train, context_test, context_val,samp, m_type, None, name_vanilla)
   ,(pca_context_train, pca_context_test, pca_context_val, samp, m_type, None ,name_pca)
   ,(autoenc_context_train, autoenc_context_test, autoenc_context_val,samp, m_type, None, name_auto)
   ,(onlypred_context_train, onlypred_context_test, onlypred_context_val,samp, m_type, None, name_only_pred)
   ,(fulltrain_context_train, fulltrain_context_test, fulltrain_context_val,samp, m_type, None, name_full_train)
   ,(context_train, context_test, context_val,samp, "FULL_NO_CONTEXT", None, name_user_item)
   ,(baseline_train, baseline_test, baseline_val ,samp, m_type,  {'dropout' : False , 'batch_norm' : False}, name_baseline)
    
]




In [1]:
import warnings
from multiprocessing import Pool

warnings.filterwarnings('ignore')

result_dicts = []

for run in tqdm(torun):
    result_dicts += [arch_testing_wrapper(run)]

# Saving & Plotting Results

In [44]:
import pickle 
order = [name_user_item, name_pca, name_auto, name_full_train, name_only_pred, name_vanilla, name_baseline]

def load_order_results():
    global order
    res = []
    for name in order:
        try:
            with open('{}{}.p'.format('../FULL_ARCH_RESULTS/CARS_DS/monday/', name), 'rb') as fp:
                res += [pickle.load(fp)]
        except FileNotFoundError:
            continue

    return res

path_to_save = '../FULL_ARCH_RESULTS/CARS_DS/paper/'



In [45]:
order_results = []
for name in order:
    for i, run in enumerate(torun):
        r_name = run[-1]
        if r_name == name:
            order_results += [result_dicts[i]]
            break
            

In [46]:
def update(name):
    global order_results, order, torun
    
    update_data = None
    
    for run in torun:
        if run[-1] == name:
            update_data = run
            break
    
    assert update_data is not None
    
    order_results[order.index(name)] = arch_testing_wrapper(update_data)


In [47]:
from datetime import datetime

try :
    path_to_save = stamp
    path_to_save = '../FULL_ARCH_RESULTS/CARS_DS/{}/'.format(stamp)
    print("Exists")
except NameError:
    print("New Stamp!")
    stamp = datetime.now().strftime("%d-%m-%Y_%I-%M-%S_%p")    
    path_to_save = '../FULL_ARCH_RESULTS/CARS_DS/{}/'.format(stamp)
    

In [48]:
def save_current_model_and_results():
    try:
        os.mkdir(path_to_save)
    except FileExistsError:
        pass

    try:
        import cPickle as pickle
    except ImportError:  # python 3.x
        import pickle

    for i, name in enumerate(order):
        data = order_results[i]    
        with open('{}{}.p'.format(path_to_save,name), 'wb') as fp:
            pickle.dump(data, fp, protocol=pickle.HIGHEST_PROTOCOL)
    from keras.utils import plot_model
    dummy_model = get_full_arch(100)

    plot_model(dummy_model, to_file='{}model.png'.format(path_to_save))

    # Open the file
    with open(path_to_save + 'report.txt','w') as fh:
        # Pass the file handle in as a lambda function to make it callable
        dummy_model.summary(print_fn=lambda x: fh.write(x + '\n'))

In [51]:
# path_to_save = '../FULL_ARCH_RESULTS/tmp/'
try:
    os.mkdir(path_to_save)
except:
    pass

In [52]:
import matplotlib.pyplot as plt
import seaborn as sns

def box_plot_methods(method_test_dict, method_val_dict, samples):
    metric = []
    methods = []
    test_val = []
    
    METRIC_VAL_LABELS = ['VAL'] * samples
    METRIC_TEST_LABELS = ['TEST'] * samples
    
    method_names = method_test_dict.keys()
    for name in method_names:
        metric += list(method_val_dict[name])
        methods += [name] * samples
        test_val += METRIC_VAL_LABELS
        
        metric += list(method_test_dict[name])
        methods += [name] * samples
        test_val += METRIC_TEST_LABELS
        

    
    print(len(metric), len(methods), len(test_val))
    df = pd.DataFrame({"METRIC" : metric , "Method" : methods , "TEST/VAL" : test_val})
    return df



In [53]:
test_avg_dict = {}
['ROC-AUC' , 'MSE', 'LOGLOSS' , 'MAE']

def iter_dict_and_update(metric_dict):
    global test_avg_dict
    
    for key in metric_dict:
        if key not in test_avg_dict:
            test_avg_dict[key] = []
        test_avg_dict[key] += [np.mean(metric_dict[key])]

In [54]:
def get_test_val_dict(val_keyword, test_keyword):
    test_dict = {}
    val_dict = {}
    for i, name in enumerate(order):
        test_dict[name] = order_results[i][test_keyword]
        val_dict[name] = order_results[i][val_keyword]
    
    return val_dict, test_dict

In [55]:
def get_best_test_val_dict(val_keyword, test_keyword):
    test_dict = {}
    val_dict = {}
    for i, name in enumerate(order):
        val_results = order_results[i][val_keyword]
        test_results = order_results[i][test_keyword]
        
        if np.mean(val_results) > np.mean(test_results):
            index = val_results.index(max(val_results))
        else:
            index = val_results.index(min(val_results))
        
        print(val_results)
        test_dict[name] = [test_results[index]]
        val_dict[name] = [val_results[index]]
    
    return val_dict, test_dict

In [2]:
val_dict, test_dict = get_test_val_dict("auc_vals", "auc_tests")
df_res = box_plot_methods(test_dict, val_dict , samp)

sns.set(style="whitegrid")
g = sns.catplot(x="TEST/VAL", y="METRIC",
                hue="Method",
                data=df_res, kind="box",
                height=7.5, aspect=1, legend=False)

g.map(plt.axvline, x=0.5, ls='--', c='black')
plt.legend(loc='center right', bbox_to_anchor=(1.4, 0.5), ncol=1)
plt.ylabel("AUC")
plt.title("CARS Dataset")
plt.savefig('{}auc_arc.png'.format(path_to_save), bbox_inches="tight")
iter_dict_and_update(test_dict)

In [3]:
val_dict, test_dict = get_test_val_dict("mse_vals", "mse_tests")

df = box_plot_methods(test_dict, val_dict , samp)

sns.set(style="whitegrid")
g = sns.catplot(x="TEST/VAL", y="METRIC",
                hue="Method",
                data=df, kind="box",
                height=7.5, aspect=1, legend=False)
g.map(plt.axvline, x=0.5, ls='--', c='black')
plt.legend(loc='center right', bbox_to_anchor=(1.4, 0.5), ncol=1)
plt.ylabel("MSE")
plt.title("CARS Dataset")
plt.savefig('{}mse_arc.png'.format(path_to_save), bbox_inches="tight")
iter_dict_and_update(test_dict)

In [4]:
val_dict, test_dict = get_test_val_dict("logloss_vals", "logloss_tests")

df = box_plot_methods(test_dict, val_dict , samp)

sns.set(style="whitegrid")
g = sns.catplot(x="TEST/VAL", y="METRIC",
                hue="Method",
                data=df, kind="box",
                height=7.5, aspect=1, legend=False)
g.map(plt.axvline, x=0.5, ls='--', c='black')
plt.legend(loc='center right', bbox_to_anchor=(1.4, 0.5), ncol=1)
plt.ylabel("logloss")
plt.title("CARS Dataset")
g.savefig("{}logloss_arc.png".format(path_to_save))
iter_dict_and_update(test_dict)

In [60]:
from sklearn.metrics import mean_absolute_error

test_dict = {}
for i, name in enumerate(order):
    test_dict[name] = order_results[i]
    
for key in test_dict.keys():
    c_dict = test_dict[key]
    preds = c_dict['y_test_pred']
    preds_val = c_dict['y_val_pred']
    
    maes_test = list(map(lambda y_pred : mean_absolute_error(y_test, y_pred), preds))
    c_dict["mae_tests"] = maes_test
    
    maes_val = list(map(lambda y_pred : mean_absolute_error(y_val, y_pred), preds_val))
    c_dict["mae_vals"] = maes_val
    
    
    
    preds = np.mean(maes_test)
    test_avg_dict[key] += [preds]

In [5]:
val_dict, test_dict = get_test_val_dict("mae_vals", "mae_tests")

df = box_plot_methods(test_dict, val_dict , samp)

sns.set(style="whitegrid")
g = sns.catplot(x="TEST/VAL", y="METRIC",
                hue="Method",
                data=df, kind="box",
                height=7.5, aspect=1, legend=False)
g.map(plt.axvline, x=0.5, ls='--', c='black')
plt.legend(loc='center right', bbox_to_anchor=(1.4, 0.5), ncol=1)
plt.ylabel("MAE")
plt.title("CARS Dataset")
g.savefig("{}mae_arc.png".format(path_to_save))
iter_dict_and_update(test_dict)

In [6]:
columns = ['AUC-ROC', 'MSE', 'LOGLOSS', 'MAE','?']
indexes = []
averages = []

for key in test_avg_dict.keys():
    indexes += [key]
    averages += [tuple(test_avg_dict[key])]
    
df_averages = pd.DataFrame(averages, columns = columns, index=indexes)
df_averages